# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [8]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, input_shape=(n_features,), activation='relu'))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 1s 8ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [14]:
# Numeric column names
numeric_columns = X_train_numeric.columns 

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [15]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [16]:
# Train the model 
normalized_input_model.fit(X_train, y_train, batch_size=32, epochs=150, validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 6ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [=============================

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [17]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [18]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))


Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.6169 - mse: 0.6169 - val_loss: 0.1798 - val_mse: 0.1798
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2239 - mse: 0.2239 - val_loss: 0.1391 - val_mse: 0.1391
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2036 - mse: 0.2036 - val_loss: 0.1239 - val_mse: 0.1239
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2006 - mse: 0.2006 - val_loss: 0.1187 - val_mse: 0.1187
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1715 - mse: 0.1715 - val_loss: 0.1190 - val_mse: 0.1190
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1725 - mse: 0.1725 - val_loss: 0.1162 - val_mse: 0.1162
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0981 - mse: 0.0981 - val_loss: 0.1186 - val_mse: 0.1186
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1380 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0275 - mse: 0.0275 - val_loss: 0.1295 - val_mse: 0.1295
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1296 - val_mse: 0.1296
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0245 - mse: 0.0245 - val_loss: 0.1286 - val_mse: 0.1286
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.1302 - val_mse: 0.1302
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1288 - val_mse: 0.1288
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0235 - mse: 0.0235 - val_loss: 0.1298 - val_mse: 0.1298
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0214 - mse: 0.0214 - val_loss: 0.1287 - val_mse: 0.1287
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0230 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.1332 - val_mse: 0.1332
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1319 - val_mse: 0.1319
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1329 - val_mse: 0.1329
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.1322 - val_mse: 0.1322
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0139 - mse: 0.0139 - val_loss: 0.1316 - val_mse: 0.1316
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.1314 - val_mse: 0.1314
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1333 - val_mse: 0.1333
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 590us/step - loss: 0.0089 - mse: 0.0089


[0.00892686378210783, 0.00892686378210783]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [20]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 750us/step - loss: 0.1335 - mse: 0.1335


[0.13352733850479126, 0.13352733850479126]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [21]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

28715.512901635917

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [22]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.7889 - mse: 0.7889 - val_loss: 0.2579 - val_mse: 0.2579
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.3343 - mse: 0.3343 - val_loss: 0.2025 - val_mse: 0.2025
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1726 - mse: 0.1726 - val_loss: 0.1689 - val_mse: 0.1689
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1716 - mse: 0.1716 - val_loss: 0.1927 - val_mse: 0.1927
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1992 - mse: 0.1992 - val_loss: 0.1502 - val_mse: 0.1502
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1379 - mse: 0.1379 - val_loss: 0.1578 - val_mse: 0.1578
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1272 - mse: 0.1272 - val_loss: 0.1395 - val_mse: 0.1395
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1416 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0268 - mse: 0.0268 - val_loss: 0.1519 - val_mse: 0.1519
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0260 - mse: 0.0260 - val_loss: 0.1536 - val_mse: 0.1536
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0254 - mse: 0.0254 - val_loss: 0.1499 - val_mse: 0.1499
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0256 - mse: 0.0256 - val_loss: 0.1530 - val_mse: 0.1530
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0251 - mse: 0.0251 - val_loss: 0.1550 - val_mse: 0.1550
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1537 - val_mse: 0.1537
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.1636 - val_mse: 0.1636
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0246 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0119 - mse: 0.0119 - val_loss: 0.1612 - val_mse: 0.1612
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1606 - val_mse: 0.1606
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0122 - mse: 0.0122 - val_loss: 0.1624 - val_mse: 0.1624
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.1615 - val_mse: 0.1615
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.1635 - val_mse: 0.1635
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0113 - mse: 0.0113 - val_loss: 0.1627 - val_mse: 0.1627
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1647 - val_mse: 0.1647
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [23]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 614us/step - loss: 0.0090 - mse: 0.0090


[0.009036987088620663, 0.009036987088620663]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 783us/step - loss: 0.1648 - mse: 0.1648


[0.16479118168354034, 0.16479118168354034]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [25]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.7072 - mse: 0.7072 - val_loss: 0.2111 - val_mse: 0.2111
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.2557 - mse: 0.2557 - val_loss: 0.1677 - val_mse: 0.1677
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1480 - mse: 0.1480 - val_loss: 0.1850 - val_mse: 0.1850
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1389 - mse: 0.1389 - val_loss: 0.1360 - val_mse: 0.1360
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1692 - mse: 0.1692 - val_loss: 0.1224 - val_mse: 0.1224
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1158 - mse: 0.1158 - val_loss: 0.1259 - val_mse: 0.1259
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0993 - mse: 0.0993 - val_loss: 0.1179 - val_mse: 0.1179
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1455 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0184 - mse: 0.0184 - val_loss: 0.1279 - val_mse: 0.1279
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0192 - mse: 0.0192 - val_loss: 0.1290 - val_mse: 0.1290
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0209 - mse: 0.0209 - val_loss: 0.1291 - val_mse: 0.1291
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0178 - mse: 0.0178 - val_loss: 0.1323 - val_mse: 0.1323
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1297 - val_mse: 0.1297
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0184 - mse: 0.0184 - val_loss: 0.1300 - val_mse: 0.1300
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0183 - mse: 0.0183 - val_loss: 0.1293 - val_mse: 0.1293
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0174 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0100 - mse: 0.0100 - val_loss: 0.1349 - val_mse: 0.1349
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1379 - val_mse: 0.1379
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0096 - mse: 0.0096 - val_loss: 0.1351 - val_mse: 0.1351
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.1342 - val_mse: 0.1342
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1356 - val_mse: 0.1356
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1360 - val_mse: 0.1360
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0084 - mse: 0.0084 - val_loss: 0.1360 - val_mse: 0.1360
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [26]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 627us/step - loss: 0.0068 - mse: 0.0068


[0.006822038441896439, 0.006822038441896439]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [27]:
# Evaluate the model on validate data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 630us/step - loss: 0.0068 - mse: 0.0068


[0.006822038441896439, 0.006822038441896439]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [28]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.3593 - mse: 0.3593 - val_loss: 0.0964 - val_mse: 0.0964
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1009 - mse: 0.1009 - val_loss: 0.1081 - val_mse: 0.1081
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1218 - mse: 0.1218 - val_loss: 0.0935 - val_mse: 0.0935
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0868 - mse: 0.0868 - val_loss: 0.0881 - val_mse: 0.0881
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0830 - mse: 0.0830 - val_loss: 0.0768 - val_mse: 0.0768
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0587 - mse: 0.0587 - val_loss: 0.0987 - val_mse: 0.0987
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0453 - mse: 0.0453 - val_loss: 0.0883 - val_mse: 0.0883
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0416 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0149 - mse: 0.0149 - val_loss: 0.0904 - val_mse: 0.0904
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0070 - mse: 0.0070 - val_loss: 0.0812 - val_mse: 0.0812
Epoch 65/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0069 - mse: 0.0069 - val_loss: 0.0853 - val_mse: 0.0853
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0051 - mse: 0.0051 - val_loss: 0.0958 - val_mse: 0.0958
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0076 - mse: 0.0076 - val_loss: 0.0967 - val_mse: 0.0967
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0080 - mse: 0.0080 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0088 - mse: 0.0088 - val_loss: 0.0917 - val_mse: 0.0917
Epoch 70/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0061 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0040 - mse: 0.0040 - val_loss: 0.0816 - val_mse: 0.0816
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.0813 - val_mse: 0.0813
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0064 - mse: 0.0064 - val_loss: 0.0896 - val_mse: 0.0896
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0075 - mse: 0.0075 - val_loss: 0.0800 - val_mse: 0.0800
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0051 - mse: 0.0051 - val_loss: 0.0810 - val_mse: 0.0810
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.0829 - val_mse: 0.0829
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0043 - mse: 0.0043 - val_loss: 0.0951 - val_mse: 0.0951
Epoch 132/150
33/33 [==============================] - 0s 2ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [29]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 584us/step - loss: 0.0017 - mse: 0.0017


[0.0017329088877886534, 0.0017329088877886534]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [30]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 724us/step - loss: 0.0777 - mse: 0.0777


[0.07767627388238907, 0.07767627388238907]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [32]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 5ms/step - loss: 0.7279 - mse: 0.7279 - val_loss: 0.1768 - val_mse: 0.1768
Epoch 2/150
33/33 [==============================] - 0s 2ms/step - loss: 0.1834 - mse: 0.1834 - val_loss: 0.1462 - val_mse: 0.1462
Epoch 3/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0989 - mse: 0.0989 - val_loss: 0.1137 - val_mse: 0.1137
Epoch 4/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0826 - mse: 0.0826 - val_loss: 0.1112 - val_mse: 0.1112
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0625 - mse: 0.0625 - val_loss: 0.1223 - val_mse: 0.1223
Epoch 6/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0485 - mse: 0.0485 - val_loss: 0.1132 - val_mse: 0.1132
Epoch 7/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0379 - mse: 0.0379 - val_loss: 0.1058 - val_mse: 0.1058
Epoch 8/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0295 - m

33/33 [==============================] - 0s 2ms/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.0999 - val_mse: 0.0999
Epoch 64/150
33/33 [==============================] - 0s 2ms/step - loss: 7.9658e-04 - mse: 7.9658e-04 - val_loss: 0.1003 - val_mse: 0.1003
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 6.4313e-04 - mse: 6.4313e-04 - val_loss: 0.1019 - val_mse: 0.1019
Epoch 66/150
33/33 [==============================] - 0s 2ms/step - loss: 7.1801e-04 - mse: 7.1801e-04 - val_loss: 0.0996 - val_mse: 0.0996
Epoch 67/150
33/33 [==============================] - 0s 2ms/step - loss: 6.8645e-04 - mse: 6.8645e-04 - val_loss: 0.1024 - val_mse: 0.1024
Epoch 68/150
33/33 [==============================] - 0s 2ms/step - loss: 4.4660e-04 - mse: 4.4660e-04 - val_loss: 0.0986 - val_mse: 0.0986
Epoch 69/150
33/33 [==============================] - 0s 2ms/step - loss: 6.1254e-04 - mse: 6.1254e-04 - val_loss: 0.1009 - val_mse: 0.1009
Epoch 70/150
33/33 [=====================

33/33 [==============================] - 0s 2ms/step - loss: 0.0072 - mse: 0.0072 - val_loss: 0.1014 - val_mse: 0.1014
Epoch 124/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0071 - mse: 0.0071 - val_loss: 0.1006 - val_mse: 0.1006
Epoch 125/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.1058 - val_mse: 0.1058
Epoch 126/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.0936 - val_mse: 0.0936
Epoch 127/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.1062 - val_mse: 0.1062
Epoch 128/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0024 - mse: 0.0024 - val_loss: 0.0969 - val_mse: 0.0969
Epoch 129/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0035 - mse: 0.0035 - val_loss: 0.1090 - val_mse: 0.1090
Epoch 130/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0036 -

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [33]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 654us/step - loss: 0.0017 - mse: 0.0017  


[0.0017012881580740213, 0.0017012881580740213]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [34]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 809us/step - loss: 0.0919 - mse: 0.0919


[0.09191825240850449, 0.09191825240850449]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [35]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 814us/step - loss: 0.1408 - mse: 0.1408


[0.14075185358524323, 0.14075185358524323]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [36]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

29482.108756798934

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.